In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

comp classify 60 secs datad thus justifing a persons state 1st and 2nd.

train.csv contains
* ~26k entries (rows)
* of 60 sec (time gaps)(subrows)
* 13 biological sensors (columns)
* for 1k experimental participants(test cases)(plus dont know how this effects the outcomes)

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
test=pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")

In [ ]:
train_labels=pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")

In [ ]:
train

In [ ]:
train.info()

In [ ]:
a=train.groupby(sort=False,by="sequence").mean()

In [ ]:
a=train.subject.unique()
a=np.array(a)
len(a)

In [ ]:
a.sort()
l=a==[i for i in range(0,672)]
if l.all():
    print("TRue")
    

In [ ]:
train_labels
train[["sequence","subject"]].iloc[60:60+60] 
#implication --> sequence corresponds to an explicit subjects
#train=train.drop("subject",axis=1)
train

In [ ]:
train_sequence=[]
train_labels

In [ ]:
for i in train.groupby(sort=False,by="sequence"):
    train_sequence.append(i)
train_sequence[1][1].head()

In [ ]:
def call(df,seq):
    return df[seq][1]
call(train_sequence,1).head()

# seperating indexes for 1's and 0's reading to understand the data

In [ ]:
train_labels
zeros_index=[]
ones_index=[]
for i in range(len(train_labels)):
    if train_labels["state"].iloc[i]==1:
        ones_index.append(i)
    else:
        zeros_index.append(i)
print(len(ones_index),len(zeros_index))

almost equal ones and zeros to train :)

# visualising senosory data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
features_sen=[i for i in train.columns if "sensor" in i]
#call(train_sequence,j)["sensor_00"]

# superimposed reading

In [ ]:
def super_imp(arr,tt=train_sequence):
    for i in features_sen:
        plt.figure(figsize=(18, 3))
        for j in arr:#add any to the bracket for comparing in one
            l=call(tt,j)
            l=l.set_index("step")
            plt.plot(l[i])#x=60,y=values of column
        plt.legend(arr)
        plt.title(str(i))
        plt.show()

# comparsion

In [ ]:
from random import shuffle

# displaying ones

In [ ]:
train_df=train.copy()
features=features_sen
shuffle(ones_index)
sequences = ones_index[:4]#[0, 1, 2, 8364, 15404,]
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train_df[train_df.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()
print(train_labels.loc[sequences,["state"]])
sequences_1=sequences

# displaying zeros

In [ ]:
train_df=train.copy()
features=features_sen
shuffle(zeros_index)
sequences_0 = zeros_index[:8]#[0, 1, 2, 8364, 15404,]
sequences=sequences_0
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train_df[train_df.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()
print(train_labels.loc[sequences_0,["state"]])

In [ ]:
super_imp(sequences_0)

pass data through a function to remove noise while maintsing integrity of the data


In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
mydata = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv', names = ['value'], header = 0) 
mydata
print(1)
#res=adfuller(mydata.value.dropna())
print(2)

#res

In [ ]:
#dickey fuller test for stationarity since applying arima model p<0.005
no_adfs=[]
for i in range(25968):
    re=adfuller(mydata["value"][i])
    if re[1]<=0.005:
        pass
    else:
        no_adfs.append(i)
    if i in [25968,int(25968*(0.75)),int(25968*(0.5)),int(25968*(0.25)),0]:
        print(i,end=" ")

In [ ]:
len(no_adfs),len(train)
len(mydata.value[0])

In [ ]:
train["index"]=range(len(train))
a=train.set_index("sequence",False).drop(no_adfs,axis=0)
b=train.set_index("index")
tt=[]
for i in b.groupby(sort=False,by="sequence"):
    tt.append(i)

In [ ]:
train_2=a

# build model

In [ ]:
#copied code for testing
def BuildNN():
    model = keras.models.Sequential([
        keras.layers.Input(shape=(60, cols)),
        keras.layers.Conv1D(32*2, 7),
        
        #keras.layers.BatchNormalization(),
        
        #keras.layers.Bidirectional(keras.layers.LSTM(700, return_sequences=True)),
        
        #keras.layers.Bidirectional(keras.layers.LSTM(700, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(768, return_sequences=True)),
        #keras.layers.Bidirectional(keras.layers.GRU(units=256,return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.GRU(units=256,return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
        
        #keras.layers.Conv1D(32*2, 7),
        keras.layers.MaxPooling1D(),
        
        keras.layers.Conv1D(64*2, 3),
        keras.layers.MaxPooling1D(),
        
        keras.layers.Conv1D(128*2, 3),
        keras.layers.GlobalMaxPooling1D(),
        
        keras.layers.Dense(150, activation="swish"),
        keras.layers.Dense(100, activation="selu"),#(swish everything else remains same)
        keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC()])
    return model

In [ ]:
def dnn_model():
    
    x_input =keras.layers.Input(shape=(60, cols))
    
    x1 = keras.layers.Bidirectional(keras.layers.LSTM(units=512, return_sequences=True))(x_input)
    x2 = keras.layers.Bidirectional(keras.layers.LSTM(units=384, return_sequences=True))(x1)
    z1 = keras.layers.Bidirectional(keras.layers.GRU(units=384, return_sequences=True))(x1)
    
    c = keras.layers.Concatenate(axis=2)([x2, z1])
    
    x3 = keras.layers.Bidirectional(keras.layers.LSTM(units=256, return_sequences=True))(c)
    x30=keras.layers.Bidirectional(keras.layers.LSTM(units=256,return_sequences=True))(x1)
    
    c1=keras.layers.Concatenate(axis=2)([x3,x30])
    #x33=keras.layers.Bidirectional(keras.layers.LSTM(units=128))
    #x31=keras.layers.Conv1D(32*2, 7)(c1)
    #x32=keras.layers.MaxPooling1D()(x31)
    
    #x33=keras.layers.Conv1D(64*2, 3)(x32)
    #x34=keras.layers.MaxPooling1D()(x33)
    #x35=keras.layers.Conv1D(128, 3)(x34)
    x4 = keras.layers.GlobalMaxPooling1D()(c1)
    x41=keras.layers.Dense(units=128,activation="selu")(x4)
    #x5 = keras.layers.Dense(units=100, activation='swish')(x41)
    x42=keras.layers.Dropout(0.05)(x41)
    x_output = keras.layers.Dense(1, activation='sigmoid')(x42)

    model = keras.models.Model(inputs=x_input, outputs=x_output, name='lstm_model')
    
    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from IPython.display import display
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

In [ ]:
'''tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)'''

# modifying feature 2 in train_2 and test

In [ ]:
train_2.head()

In [ ]:
a=train_2.corr()
import seaborn as sns
plt.figure(figsize=(12,12))
sns.heatmap(a,annot=True)
R_2_n=[]
for i in range(4,len(a["sensor_02"])):
    if (a["sensor_02"][i]**2)**(0.5)>0.001:
        R_2_n.append(i-4)
R_2_n=R_2_n[:-1]
R_2_n

In [ ]:
s_2_n=[list(train_2["sensor_02"])]+[list(train_2["sensor_0"+str(i)]) for i in R_2_n]
trial=train_2.copy()
k=list(np.array(s_2_n[0])*(np.array(s_2_n[1])+(0.2)*np.array(s_2_n[2])))
trial["sensor_02"]=k
#/np.array(s_2_n[3])))
plt.figure(figsize=(12,13))
sns.heatmap(trial.corr(),annot=True)
trial

GOOD CORR IN 2 OBTAINED ,THERFORE NOT GOING TO DROP SENS-2 

In [ ]:
'''train_2["sensor_02"]=k
s_2_n=[list(test["sensor_02"])]
s_2_n+=[list(test["sensor_0"+str(i)]) for i in R_2_n]
test["sensor_02"]=list(np.array(s_2_n[0])*(np.array(s_2_n[1])+(0.2)*np.array(s_2_n[2])))
train_2.head()'''

not doing any cahnges with sensor 2 as it leads to over fitting


# Adding other features


In [ ]:
train_2.head()

In [ ]:
train_2=train_2.set_index("index")
features = train_2.columns.tolist()[3:]
train_2

In [ ]:
def prep(df):
    for feature in features:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1)
        df.fillna(0, inplace=True)
        df[feature + '_diff1'] = df[feature] - df[feature + '_lag1'] 
prep(train_2)
prep(test)

In [ ]:
#sumation of different columns
s=train_2["sensor_00"]+train_2["sensor_01"]+train_2["sensor_06"]+train_2["sensor_09"]
s1=train_2["sensor_02"]+train_2["sensor_03"]+train_2["sensor_07"]+train_2["sensor_12"]
s2=train_2["sensor_08"]+train_2["sensor_04"]+train_2["sensor_05"]
d=s-s1
train_2["sum_1"]=s
train_2["sum_2"]=s1
train_2["sum_3"]=s2
train_2["diff"]=d
#train_2["diff1"]=s-s2
plt.figure(figsize=(16,16))
sns.heatmap(train_2.corr(),annot=True)

good results converting test too

In [ ]:
s=test["sensor_00"]+test["sensor_01"]+test["sensor_06"]+test["sensor_09"]
s1=test["sensor_02"]+test["sensor_03"]+test["sensor_07"]+test["sensor_12"]
s2=test["sensor_08"]+test["sensor_04"]+test["sensor_05"]
d=s-s1
test["sum_1"]=s
test["sum_2"]=s1
test["sum_3"]=s2
test["diff"]=d
#test["diff1"]=s-s2

In [ ]:
train_2.head()
test.head()

# scalling data(standardization)

In [ ]:
cul=train_2.columns
cols=len(cul)-3
cols

In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler

sc =StandardScaler() #RobustScaler()
b=sc.fit_transform(train_2)
a=sc.transform(test)
a=pd.DataFrame(a)
b=pd.DataFrame(b)
test=a.set_axis(cul,axis="columns")
train_2=b.set_axis(cul,axis="columns")

In [ ]:
plt.figure(figsize=(19,19))
sns.heatmap(train_2.corr(),annot=True)

In [ ]:
train=train_2
groups = train["sequence"]
train = train.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
test = test.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
labels = train_labels["state"]
labels=labels.drop(no_adfs,axis=0)

# ensemble

In [ ]:
"""def get_score(model,Xtr,Xt,ytr,yt):
    model.fit(Xtr,ytr)
    pred=model.predict(Xt)
    return roc_auc_score(yt, pred)"""

In [ ]:
"""from xgboost import XGBRegressor as x
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for train_i,test_i in kf.split(list(train.index)):
    Xtr=train.iloc[list(train_i)]
    Xt=train.iloc[list(test_i)]
    ytr=labels.iloc[list(train_i)]
    yt=labels.iloc[list(test_i)]
    model=x()
    print(get_score(model,Xtr,Xt,ytr,yt),get_score(model,train[features],Xt,train[target],yt))"""

# bi-lstm rNN training

In [ ]:
train = train.reshape(int(len(train)/60), 60, cols)
test = test.reshape(int(len(test)/60), 60, cols)

In [ ]:
train.shape
#len(train)

In [ ]:
cv_score = 0
test_preds = []
kf = GroupKFold(n_splits=5)
for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(train, labels, groups.unique())):
    
    print("*"*15, f"Fold {fold_idx+1}", "*"*15)
    
    X_train, X_valid = train[train_idx], train[valid_idx]
    y_train, y_valid = labels.iloc[train_idx].values, labels.iloc[valid_idx].values
    
    lr = keras.callbacks.ReduceLROnPlateau(monitor="val_auc", factor=0.5,patience=5, verbose=True)
    es = keras.callbacks.EarlyStopping(monitor="val_auc", patience=8,verbose=True, mode="max",restore_best_weights=True)
    model = dnn_model()#dnn_model()
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics='AUC')
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=256,verbose=True, 
              callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),lr,es])
    
    cv_score += roc_auc_score(y_valid, model.predict(X_valid).squeeze())
    print(cv_score/(1+fold_idx),roc_auc_score(labels, model.predict(train)))
    print("{_|_|_}")
    test_preds.append(model.predict(test).squeeze())
    
print(cv_score/5)

In [ ]:
a=roc_auc_score(labels[int(len(labels)/1000):int(len(labels)/1)], model.predict(train[int(len(labels)/1000):int(len(labels)/1)]))

In [ ]:
a

In [ ]:
submission=pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")

In [ ]:
submission["state"] = sum(test_preds)/(5)
ans=[]
for i in submission["state"]:
    ans.append(i)
y=[i/100 for i in range(0,len(ans))]
#plt.figure(figsize=(12,12))
sns.histplot(ans,kde=True)

In [ ]:
submission["state"] =ans

In [ ]:
submission.to_csv("submission.csv", index=False)
submission

27)
no/cum/allover
1. 0.964270 0.986066
2. 0.963056 0.986797
3. 0.963164 0.986134
4. 0.962730 0.984839
5. 0.962141 0.985073

28)no/cum/allover
1. 0.966718 0.985501
2. 0.964210 0.97889
3. 0.962745 0.984175
4. 0.962804 0.983168
5. 0.961684 0.97622


29) 0.958(recent)
1. 0.964171 0.989094
2. 0.965284 0.990147
3. 0.965110 0.988886
4. 0.965273 0.989805
5. 0.963674 0.989546
